In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [2]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies(원핫인코딩), crosstab
import seaborn as sns # iris 데이터(데이터프레임) 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [25]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                   test_size=0.2,
                                                   stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [26]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=50, activation='relu'),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 64)                320       
                                                                 
 dense_29 (Dense)            (None, 128)               8320      
                                                                 
 dense_30 (Dense)            (None, 50)                6450      
                                                                 
 dense_31 (Dense)            (None, 3)                 153       
                                                                 
Total params: 15,243
Trainable params: 15,243
Non-trainable params: 0
_________________________________________________________________


In [32]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,
               epochs=200,
               validation_split=0.1,
               callbacks=[earlystopping])

Epoch 1/200
4/4 [==============================] - 1s 53ms/step - loss: 0.0931 - accuracy: 0.9537 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 2/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0750 - accuracy: 0.9722 - val_loss: 0.0781 - val_accuracy: 0.9167
Epoch 3/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0825 - accuracy: 0.9352 - val_loss: 0.0326 - val_accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0558 - accuracy: 0.9815 - val_loss: 0.0119 - val_accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0735 - accuracy: 0.9722 - val_loss: 0.0201 - val_accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0609 - accuracy: 0.9907 - val_loss: 0.0342 - val_accuracy: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0620 - accuracy: 0.9907 - val_loss: 0.0199 - val_accuracy: 1.0000
Epoch 8/200
4/4 [====

4/4 [==============================] - 0s 10ms/step - loss: 0.0633 - accuracy: 0.9815 - val_loss: 0.0183 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0705 - accuracy: 0.9722 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0705 - accuracy: 0.9722 - val_loss: 0.0281 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0563 - accuracy: 0.9907 - val_loss: 0.0388 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0566 - accuracy: 0.9815 - val_loss: 0.0183 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0555 - accuracy: 0.9815 - val_loss: 0.0121 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0671 - accuracy: 0.9815 - val_loss: 0.0215 - val_accuracy: 1.0000
Epoch 66/200
4/4 [=============

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [28]:
from sklearn.neural_network import MLPClassifier

In [29]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [33]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver='adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습 최대 횟수
            early_stopping=True, # 조기 종료 활성화
            n_iter_no_change=20, # earlyStopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start=False # True일 경우 이전학습에 이어서 학습
)

In [38]:
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50), max_iter=1000,
              n_iter_no_change=20)

In [39]:
model.score(test_X, test_y)

0.8

In [40]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [42]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [45]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X)
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,4,6
2,0,0,10


# 3. 클래스를 생성하여 모델 생성함수 사용

In [46]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(units=50, activation=activation))
#         model.add(Dense(units=30, activation=activation))
#         model.add(Dense(units=3, activation='softmax'))
        # 모델구성
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=output_dim, activation='softmax')
        ])
        # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model        

In [55]:
# 1. 데이터
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델 학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1, verbose=0)

(120, 4) (120,) (30, 4) (30,)


In [57]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100} %')

1/1 [==============================] - 0s 20ms/step - loss: 0.2006 - accuracy: 0.9667
정확도 : 96.66666388511658 %


# 4. 함수형 API
- 병렬처리, Residual block

In [61]:
# 기존 model 스타일1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
# 기존 model 스타일2
model = Sequential([
    Input(shape=(4,)),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')
])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 50)                250       
                                                                 
 dense_52 (Dense)            (None, 30)                1530      
                                                                 
 dense_53 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [59]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 4)]               0         
                                                                 
 dense_45 (Dense)            (None, 50)                250       
                                                                 
 dense_46 (Dense)            (None, 30)                1530      
                                                                 
 dense_47 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [63]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_58 (Dense)               (None, 50)           250         ['input_18[0][0]']               
                                                                                                  
 dense_59 (Dense)               (None, 80)           400         ['input_18[0][0]']               
                                                                                                  
 dense_60 (Dense)               (None, 30)           150         ['input_18[0][0]']               
                                                                                            

In [68]:
# 리지듀얼 블얼(Residual Block)
    #딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add
input_ = Input(shape=(4, ))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2])
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_68 (Dense)               (None, 50)           250         ['input_21[0][0]']               
                                                                                                  
 dense_69 (Dense)               (None, 50)           2550        ['dense_68[0][0]']               
                                                                                                  
 add_2 (Add)                    (None, 50)           0           ['dense_68[0][0]',               
                                                                  'dense_69[0][0]']         

In [69]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=0)

In [70]:
# 학습 평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9666666388511658
